In [1]:
import pandas as pd

In [2]:
web_path = "../data/Web_Traffic.csv"
web_df = pd.read_csv(web_path, parse_dates=["Visit Date"])
web_df.head()

/tmp/ipykernel_12474/1231117421.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  web_df = pd.read_csv(web_path, parse_dates=["Visit Date"])


,Session ID,Pageviews,Visit Date,Traffic Source,Actions,Time spent (in minutes),Page Title,Contact ID,Primary Topic
0,11002649,4,2023-01-01,LinkedIn,Download,9,Contact,00v2000000KM7EdAAL,Fund Universe
1,11002963,5,2023-01-01,Organic,Srcoll 50%,13,Contact,00v2000000KLfqsAAD,Fund Universe
2,11000521,7,2023-01-01,Google,Srcoll 50%,3,Home,00v2000000KO68GAAT,Fund Universe
3,11001502,5,2023-01-01,LinkedIn,Download,14,Contact,00v2000000KrZiaAAF,Fund Universe
4,11001448,6,2023-01-01,Organic,Login,3,Home,00v2000000ez1ucAAA,Fund Universe


In [3]:
## Visit Date cahnged to correct date format

web_df["Visit Date"] = web_df["Visit Date"].astype(str)

In [4]:
## convert data type of session ID to string

web_df = web_df.astype({"Session ID": str})
web_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 428332 entries, 0 to 428331
Data columns (total 9 columns):
 #   Column                   Non-Null Count   Dtype 
---  ------                   --------------   ----- 
 0   Session ID               428332 non-null  object
 1   Pageviews                428332 non-null  int64 
 2   Visit Date               428332 non-null  object
 3   Traffic Source           428174 non-null  object
 4   Actions                  428332 non-null  object
 5   Time spent (in minutes)  428332 non-null  int64 
 6   Page Title               428332 non-null  object
 7   Contact ID               428332 non-null  object
 8   Primary Topic            291885 non-null  object
dtypes: int64(2), object(7)
memory usage: 29.4+ MB


In [5]:
## Make a new unique session ID

web_df["Session ID"] = web_df["Session ID"] + web_df["Contact ID"]

In [6]:
## correct Traffic Source column
## fill null values of Traffic Source column

web_df["Traffic Source"] = web_df["Traffic Source"].fillna("Unknown")

traffic_source_map = {
    "organic": "Organic",
    "Youtube": "YouTube"
}

web_df["Traffic Source"] = web_df["Traffic Source"] \
    .map(lambda x: x if x not in traffic_source_map.keys() else traffic_source_map[x])

web_df["Traffic Source"].value_counts()

Traffic Source
LinkedIn    179156
Organic      95363
YouTube      92733
Google       60922
Unknown        158
Name: count, dtype: int64

In [7]:
web_df["Traffic Source"].value_counts()

Traffic Source
LinkedIn    179156
Organic      95363
YouTube      92733
Google       60922
Unknown        158
Name: count, dtype: int64

In [8]:
## correct Page Title column

page_title_map = {
    "funds": "Funds"
}

web_df["Page Title"] = web_df["Page Title"] \
    .map(lambda x: x if x not in page_title_map.keys() else page_title_map[x])

web_df["Page Title"].value_counts()

Page Title
Funds         119754
Home          102632
Navigation     85356
Strategy       51686
Contact        34595
Invest         34309
Name: count, dtype: int64

In [9]:
## remove test entry

web_df.drop(web_df[web_df["Primary Topic"] == "test"].index,
            inplace=True
           )

In [10]:
web_df[web_df["Primary Topic"] == "test"]

,Session ID,Pageviews,Visit Date,Traffic Source,Actions,Time spent (in minutes),Page Title,Contact ID,Primary Topic


In [11]:
## Correct Primary Topic column

web_df["Primary Topic"] = web_df["Page Title"] 

In [12]:
web_df[["Page Title", "Primary Topic"]].value_counts()

Page Title  Primary Topic
Funds       Funds            119754
Home        Home             102631
Navigation  Navigation        85356
Strategy    Strategy          51686
Contact     Contact           34595
Invest      Invest            34309
Name: count, dtype: int64

In [13]:
web_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 428331 entries, 0 to 428331
Data columns (total 9 columns):
 #   Column                   Non-Null Count   Dtype 
---  ------                   --------------   ----- 
 0   Session ID               428331 non-null  object
 1   Pageviews                428331 non-null  int64 
 2   Visit Date               428331 non-null  object
 3   Traffic Source           428331 non-null  object
 4   Actions                  428331 non-null  object
 5   Time spent (in minutes)  428331 non-null  int64 
 6   Page Title               428331 non-null  object
 7   Contact ID               428331 non-null  object
 8   Primary Topic            428331 non-null  object
dtypes: int64(2), object(7)
memory usage: 32.7+ MB


In [14]:
# join contact dataframe with email campaign data

contact_df = pd.read_csv("../cleaned_data/contact_details_cleaned.csv")

web_df = web_df.merge(contact_df,
                      how="inner",
                      left_on = "Contact ID",
                      right_on = "Contact_id"
                     )
web_df.drop(["Contact_id"], axis=1, inplace=True)

In [15]:
clean_web_path = "../cleaned_data/web_campaign_cleaned.csv"
web_df.to_csv(clean_web_path, index=False)